<a href="https://colab.research.google.com/github/Anakha-s/Project-1/blob/main/eg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## EDA Pkgs
import pandas as pd
#Data Viz Pkg
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import numpy as np # linear algebra
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords

In [ ]:
# Load Dataset
df = pd.read_csv("covid19_tweets.csv")
# Preview
df.head(3)

In [ ]:
!pip install neattext
# Load Text Cleaning Package
import neattext.functions as nfx

In [ ]:
# Cleaning Text
df['clean_tweet'] = df['text'].apply(nfx.remove_hashtags)
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x))
# Cleaning Text: Multiple WhiteSpaces
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_multiple_spaces)
# Cleaning Text : Remove urls
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_urls)
# Cleaning Text: Punctuations
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_puncts)
#Remove blank rows if any.
df['clean_tweet'] .dropna(inplace=True)
# Change all the text to lower case.
df['clean_tweet']  = [entry.lower() for entry in df['clean_tweet'] ]
df[['text','clean_tweet']]

In [ ]:
# Tokenization
df['clean_tweet']= [word_tokenize(entry) for entry in df['clean_tweet']]
df['clean_tweet']



In [ ]:
df['clean_tweet'].head(1)

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(words):
  return [word for word in words if  not word in stop_words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: remove_stopwords(x))


def stemming(words):
  ps=PorterStemmer()
  return [ps.stem(word) for word in words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: stemming(x))

def lemmatizing(words):
    lemmatizer =WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: lemmatizing(x))

def final_text(words):
     return ' '.join(words)
df['clean_tweet']=df['clean_tweet'].apply(lambda x:final_text(x))
   


In [ ]:
df['clean_tweet'].head(1)

In [ ]:
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    sentiment_subjectivity = blob.sentiment.subjectivity
    if sentiment_polarity > 0:
        sentiment_label = 1
    elif sentiment_polarity < 0:
        sentiment_label = -1
    else:
        sentiment_label = 0
    result = {'polarity':sentiment_polarity,
              'subjectivity':sentiment_subjectivity,
              'sentiment':sentiment_label}
    return result

In [ ]:
df['sentiment_results'] = df['clean_tweet'].apply(get_sentiment)
df['sentiment_results']

In [ ]:
df = df.join(pd.json_normalize(df['sentiment_results']))
df.head(3)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['clean_tweet'],df['sentiment'],test_size=0.3)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5)
Tfidf_vect.fit(df['clean_tweet'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary_)

In [ ]:
print(Train_X_Tfidf)

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Test_Y,predictions_SVM)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Test_Y,predictions_SVM))

In [ ]:
SVCmodel = LinearSVC()
SVCmodel.fit(X_train, y_train)
model_Evaluate(SVCmodel)
y_pred2 = SVCmodel.predict(X_test)